In [ ]:
# Install blosc2 and caterva2 in Pyodide environments
import sys
if sys.platform == "emscripten":
    # Install latest blosc2
    import requests
    import micropip
    url = "https://api.github.com/repos/Blosc/python-blosc2/releases/latest"
    resp = requests.get(url, headers={"Accept": "application/vnd.github+json"})
    latest_tag = resp.json()["tag_name"].strip('v')
    wheel_name=f"blosc2-{latest_tag}-cp312-cp312-pyodide_2024_0_wasm32.whl"
    wheel_url=f"https://blosc.github.io/python-blosc2/wheels/{wheel_name}"
    await micropip.install(wheel_url)
    print(f"Installed {wheel_name} successfully!")
    
    # Install latest Caterva2
    latest_url = "https://ironarray.github.io/Caterva2/wheels/latest.txt"
    # Fetch wheel name
    wheel_name = requests.get(latest_url).text.strip()
    # Install the wheel dynamically
    wheel_url = f"https://ironarray.github.io/Caterva2/wheels/{wheel_name}"
    await micropip.install(wheel_url)
    print(f"Installed {"Caterva2-main"} successfully!")

In [ ]:
import blosc2
import caterva2 as cat2

### Caterva2: remote and seamless data management and computation

In [ ]:
%%time
# c = cat2.Client("https://cat2.cloud/demo")
c = cat2.Client("https://cat2.cloud/demo", ("user@example.com", "foobar11"))
myroot = c.get("@public")
dset = myroot["large/gaia-3d.b2nd"]
print(type(dset), dset.shape, dset.dtype)

In [ ]:
%%time
# Get stars in 100 light-years cube around our Sun
dslice = dset[9_950:10_050, 9_950:10_050, 9_950:10_050]
print("Retrieved slice shape", dslice.shape)
print("Total stars near us (computed locally):", dslice.sum())

In [ ]:
%%time
# Same, but doing the computation in the server (only works when authenticated)
lexpr = blosc2.lazyexpr('dset[9_950:10_050, 9_950:10_050, 9_950:10_050].sum()')
sum_path = c.upload(remotepath='@personal/sum_dslice.b2nd', local_dset=lexpr)
dsum = c.fetch(sum_path)
print("Total stars near us (computed remotely):", dsum[()])